In [4]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

pd.options.display.max_columns = 999

# Utah

## VEST Documentation

Election results from OpenElections (https://github.com/openelections/openelections-data-ut/tree/master/2020).
Precinct shapefile from Utah Automated Geographic Reference Center (https://gis.utah.gov/data/political/voter-precincts/).

The shapefile from the AGRC is of subprecincts, breaking down precincts in cases of district splits. In some cases, results are reported at the subprecinct level, in most cases, they weren't, so merging was done where necessary.

Emery County - 52 "canvas votes" were distributed to precincts.

G20PRERTRU - Donald J. Trump (Republican Party)  
G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
G20PREGHAW - Howie Hawkins (Green Party)  
G20PRECBLA - Don Blankenship (Constitution Party)  
G20PREIPIE - Brock Pierce (Unaffiliated)  
G20PREIWES - Kanye West (Unaffiliated)  
G20PREIMCH - Joe McHugh (Unaffiliated)  
G20PREILAR - Gloria La Riva (Unaffiliated)  
G20PREOWRI - Write-in Votes  

G20GOVRCOX - Spencer J. Cox (Republican Party)  
G20GOVDPET - Chris Peterson (Democratic Party)  
G20GOVLCOT - Daniel Rhead Cottam (Libertarian Party)  
G20GOVADUE - Greg Duerden (Independent American Party)  
G20GOVOWRI - Write-in Votes  

G20ATGRREY - Sean D. Reyes (Republican Party)  
G20ATGDSKO - Greg Skordas (Democratic Party)  
G20ATGLBAU - Rudy J. Bautista (Libertarian Party)  

G20AUDRDOU - John "Frugal" Dougall (Republican Party)  
G20AUDCOST - Jeffrey L. Ostler (Constitution Party)  
G20AUDUFAB - Brian L. Fabbi (United Utah Party)  

G20TRERDAM - David Damschen (Republican Party)  
G20TRELSPE - Joseph Speciale (Libertarian Party)  
G20TRECPRO - Richard Proctor (Constitution Party)  

## Load VEST File

In [2]:
vest_ut_20 = gp.read_file("./raw-from-source/VEST/ut_2020/ut_2020.shp")

In [3]:
#Create a list of data columns
data_columns = [col for col in vest_ut_20.columns if "G20" in col]

## Load Election Results